In [2]:
import pandas as pd

file_path2='/Users/kkkg0829/Desktop/project/data set/B0005_with_lowess_features.csv'

df=pd.read_csv(file_path2)

In [3]:
import numpy as np
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

# 1. 사용할 feature 선택
features1 = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load', 'Voltage_measured_smooth',
    'Voltage_measured_residual', 'Voltage_measured_trend',
    'Current_measured_smooth', 'Current_measured_residual',
    'Current_measured_trend', 'Temperature_measured_smooth',
    'Temperature_measured_residual', 'Temperature_measured_trend',
    'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
    'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'
]

X = df[features1]

# 2. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lof = LocalOutlierFactor(
    n_neighbors=30,
    contamination='auto' # 예상 이상치 비율
)

y_pred = lof.fit_predict(X_scaled)
lof_scores = -lof.negative_outlier_factor_

# threshold 설정 (직접 계산한 값 사용)
threshold = 1.676333654209548

df['lof_scores'] = lof_scores              # 실제 LOF score
df['lof_anomaly'] = (df['lof_scores'] > threshold).astype(int)  # 0/1로 변환

print("LOF 이상치:", df['lof_anomaly'].sum())


LOF 이상치: 638


기존 LOF baseline 모델(contamination=0.05)에서 결정된 threshold(1.6)를

고정된 기준으로 삼아,

LOWESS trend/residual 기반 features를 추가했을 때

동일 기준에서 anomaly detection sensitivity가 증가하는지 비교하였다.

결과적으로 638개의 anomaly window가 탐지되어, 파생변수 추가가 단발성 스파이크 및 국소 패턴 이상 탐지에 효과적임을 확인했다.

In [4]:
import plotly.express as px

cycle_df = df.groupby('cycle_idx')['lof_scores'].mean().reset_index()

fig = px.line(
    cycle_df,
    x='cycle_idx',
    y='lof_scores',      
    title='Average LOF Score per Cycle',
    markers=True
)
fig.show()

In [5]:
cycle_anom = df.groupby('cycle_idx')['lof_anomaly'].sum().reset_index()

fig = px.bar(
    cycle_anom,
    x='cycle_idx',
    y='lof_anomaly',
    title='Number of Anomalies per Cycle'
)
fig.show()


In [6]:
cycle_peak = df.groupby('cycle_idx')['lof_scores'].max().reset_index()
cycle_peak['is_anomaly'] = (cycle_peak['lof_scores'] > threshold).astype(int)

fig = px.line(
    cycle_peak,
    x='cycle_idx',
    y='lof_scores',
    title='Peak LOF Score per Cycle',
    markers=True
)
fig.show()

In [7]:
anom_cycles=df[df['lof_anomaly']==1]['cycle_idx'].unique()
print('Anomalic cycles:', anom_cycles)
print('number of Anomalic cycles:', len(anom_cycles))

Anomalic cycles: [  4   6   8  10  14  16  18  20  25  27  35  37  46  54  62  70  82  86
  90  94  98 102 106 110 114 118 122 130 134 136 144 148 150 154 158 166
 170 174 182 186 190 194 198 202 206 210 214 216 220 224 228 232 236 240
 244 248 252 256 260 264 270 274 278 286 290 294 298 302 306 316 320 324
 328 332 336 348 364 367 371 383 391 395 399 411 431 437 441 445 457 465
 469 473 477 481 483 485 493 509 517 525 529 533 537 541 545 548 556 564
 576 580 584 588 592 596 600 604 608 612 614]
number of Anomalic cycles: 119


In [8]:
cycle_anomaly_ratio = df.groupby('cycle_idx')['lof_anomaly'].mean().reset_index() # 이상 윈도우비율
# cycle_anomaly_ratio
cycle_anomaly_ratio['is_cycle_anomaly'] = (cycle_anomaly_ratio['lof_anomaly'] > 0.1).astype(int)
print("이상치윈도우가 0.1 초과인 싸이클 수:", cycle_anomaly_ratio['is_cycle_anomaly'].sum())
print("이상치윈도우가 0.1 초과인 싸이클 번호:", cycle_anomaly_ratio[cycle_anomaly_ratio['is_cycle_anomaly']==1]['cycle_idx'].values)

이상치윈도우가 0.1 초과인 싸이클 수: 4
이상치윈도우가 0.1 초과인 싸이클 번호: [134 150 154 614]


윈도우 기반 LOF는 단발성 이상이나 noise에 매우 민감하게 반응하여 fine-grain(아주 작고 세밀한 단위를 봄) 이상 패턴을 탐지한다.”

“그러나 cycle-level health anomaly를 판단하기 위해서는 이상 window 비율 기반의 2차 판단이 필요하다.”

“603개 window anomaly 중 cycle anomaly threshold(10%)를 통과한 cycle은 단 4개였다.”

“이는 변동성 신호는 곳곳에서 포착되지만 실제 열화(cycle-level structural degradation)는 일부 cycle에서만 나타난다는 것을 의미한다.

In [9]:
cycle_anomaly_ratio = df.groupby('cycle_idx')['lof_anomaly'].mean().reset_index() # 이상 윈도우비율
# cycle_anomaly_ratio
cycle_anomaly_ratio['is_cycle_anomaly'] = (cycle_anomaly_ratio['lof_anomaly'] > 0.05).astype(int)
print("이상치윈도우가 0.05 초과인 싸이클 수:", cycle_anomaly_ratio['is_cycle_anomaly'].sum())
print("이상치윈도우가 0.05 초과인 싸이클 번호:", cycle_anomaly_ratio[cycle_anomaly_ratio['is_cycle_anomaly']==1]['cycle_idx'].values)

이상치윈도우가 0.05 초과인 싸이클 수: 4
이상치윈도우가 0.05 초과인 싸이클 번호: [134 150 154 614]


In [10]:
# 히스토그램으로 점수 분포 겹쳐서 그리기
import plotly.express as px

fig_score = px.histogram(df, x='lof_scores', color='lof_anomaly',
                         nbins=100,
                         title='LOF Score 분포: 정상(0) vs 이상(1)',
                         labels={'lof_anomaly': 'Label (0:Normal, 1:Anomaly)'},
                         opacity=0.7,
                         log_y=True) # 데이터 개수 차이가 크므로 y축을 로그 스케일로
fig_score.show()

In [11]:
# Boxplot으로 잔차 분포 비교
fig_res = px.box(df, x='lof_anomaly', y='Voltage_measured_residual',
                 color='lof_anomaly',
                 title='정상 vs 이상: 전압 잔차(Residual) 크기 비교',
                 points='outliers') # 모든 점을 찍지 않고 이상치만 점으로 표시
fig_res.show()

In [12]:
# Cycle별 이상치 빈도 (Aging Correlation)
cycle_anomaly_counts = df[df['lof_anomaly'] == 1]['cycle_idx'].value_counts().sort_index()

fig_cycle = px.bar(
    x=cycle_anomaly_counts.index,
    y=cycle_anomaly_counts.values,
    title='<b>[Temporal Trend] Anomaly Frequency per Cycle</b>',
    labels={'x': 'Cycle Index', 'y': 'Anomaly Count'},
    color=cycle_anomaly_counts.values,
    color_continuous_scale='Viridis'
)

fig_cycle.update_layout(xaxis_title="Battery Aging (Cycle)", yaxis_title="Number of Anomalies")
fig_cycle.show()

In [13]:
import plotly.graph_objects as go

# 134번 사이클(이상치 폭발) vs 86번 사이클(정상) 비교
# 1. 비교 대상 자동 선택
# (1) 이상치 사이클: 사용자가 지정한 134번
bad_cycle_idx = 134

# (2) 정상 사이클: 데이터 개수가 가장 많은(끊김 없는) 정상 사이클 하나를 자동 선택
# 정상 데이터(Label 0) 중에서 사이클별 데이터 개수를 센다
normal_counts = df[df['lof_anomaly'] == 0]['cycle_idx'].value_counts()
# 가장 데이터가 풍부한 사이클 번호를 가져옵니다.
normal_cycle_idx = normal_counts.index[0]

print(f"=== 비교 대상 선정 ===")
print(f"이상치 사이클: {bad_cycle_idx}번")
print(f"정상 사이클 (자동선택): {normal_cycle_idx}번")

# 2. 데이터 추출
cycle_bad = df[df['cycle_idx'] == bad_cycle_idx]
cycle_normal = df[df['cycle_idx'] == normal_cycle_idx]

# 데이터 개수 확인 (혹시 0개인지 체크)
print(f" - 이상치 데이터 개수: {len(cycle_bad)}개")
print(f" - 정상 데이터 개수: {len(cycle_normal)}개")

# 3. 그래프 그리기
if len(cycle_normal) > 0 and len(cycle_bad) > 0:
    fig_compare = go.Figure()

    # 정상 사이클 (파란색 실선)
    fig_compare.add_trace(go.Scatter(
        x=np.arange(len(cycle_normal)),
        y=cycle_normal['Voltage_measured'],
        mode='lines',
        name=f'Normal Cycle ({normal_cycle_idx})',
        line=dict(color='blue', width=2),
        opacity=0.7
    ))

    # 이상치 사이클 (빨간색 점선)
    fig_compare.add_trace(go.Scatter(
        x=np.arange(len(cycle_bad)),
        y=cycle_bad['Voltage_measured'],
        mode='lines',
        name=f'Anomaly Cycle ({bad_cycle_idx})',
        line=dict(color='red', width=3, dash='dot') # 두께를 키움
    ))

    fig_compare.update_layout(
        title=f'<b>[Comparison] Normal vs Anomaly (Cycle {normal_cycle_idx} vs {bad_cycle_idx})</b>',
        xaxis_title='Time Step (Sequence)',
        yaxis_title='Voltage (V)',
        template='plotly_white',
        hovermode="x unified"
    )

    fig_compare.show()
else:
    print("오류: 선택된 사이클에 데이터가 없습니다.")

=== 비교 대상 선정 ===
이상치 사이클: 134번
정상 사이클 (자동선택): 86번
 - 이상치 데이터 개수: 179개
 - 정상 데이터 개수: 371개


In [14]:
# 누적 이상치 그래프 (Cumulative Anomaly Count)
# 시간이 지날수록 이상치가 얼마나 '쌓이는가'를 봅니다.

# 사이클별 이상치 개수 집계 (없는 사이클은 0으로 채움)
all_cycles = pd.DataFrame({'cycle_idx': df['cycle_idx'].unique()})
anomaly_counts = df[df['lof_anomaly'] == 1]['cycle_idx'].value_counts().reset_index()
anomaly_counts.columns = ['cycle_idx', 'count']

# 병합 및 정렬
trend_df = pd.merge(all_cycles, anomaly_counts, on='cycle_idx', how='left').fillna(0)
trend_df = trend_df.sort_values('cycle_idx')

# 누적 합 계산
trend_df['cumulative_count'] = trend_df['count'].cumsum()

# 시각화
fig_trend = px.area(
    trend_df,
    x='cycle_idx',
    y='cumulative_count',
    title='<b>[Cumulative Trend] Total Anomalies Over Time</b>',
    labels={'cumulative_count': 'Accumulated Anomalies', 'cycle_idx': 'Cycle Index'}
)

fig_trend.update_layout(template='plotly_white')
fig_trend.show()

In [15]:
# Feature Contribution (원인 분석)
# 로직: (이상치 그룹의 평균 - 정상 그룹의 평균) / 데이터 전체의 표준편차
# 의미: "이 피처가 정상보다 몇 시그마(Standard Deviation)만큼 벗어나 있는가?"

# 데이터 분리
normal_df = df[df['lof_anomaly'] == 0][features1]
anomaly_df = df[df['lof_anomaly'] == 1][features1]

# 차이 계산 (Z-score 관점의 차이)
diff_series = (anomaly_df.mean() - normal_df.mean()) / df[features1].std()
diff_df = diff_series.reset_index()
diff_df.columns = ['Feature', 'Deviation_Score']
diff_df = diff_df.sort_values(by='Deviation_Score', key=abs, ascending=True) # 절대값 크기순 정렬

# 시각화
fig_reason = px.bar(
    diff_df,
    x='Deviation_Score',
    y='Feature',
    orientation='h',
    title='<b>이상치 원인분석 (Feature Deviation)</b>',
    color='Deviation_Score',
    color_continuous_scale='RdBu_r', # 빨강: 높아서 문제, 파랑: 낮아서 문제
    text_auto='.2f'
)

fig_reason.add_vline(x=0, line_width=2, line_color='black')
fig_reason.update_layout(height=800)
fig_reason.show()

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
import plotly.graph_objects as go


# 1. Train/Validation/Test 분할 (6:2:2)

# 고유 cycle 개수
total_cycles = df['cycle_idx'].nunique()

# 6:2:2 split 기준 cycle 개수
train_cycles = int(total_cycles * 0.6)
val_cycles = int(total_cycles * 0.8)  # train + val

# 정렬된 cycle 리스트
cycle_list = sorted(df['cycle_idx'].unique())

# 기준 cycle 번호
train_threshold_cycle = cycle_list[train_cycles - 1]
val_threshold_cycle = cycle_list[val_cycles - 1]

print(f"총 Cycle 수: {total_cycles}")
print(f"Train Cycles: 0 ~ {train_threshold_cycle} ({train_cycles}개)")
print(f"Validation Cycles: {train_threshold_cycle+1} ~ {val_threshold_cycle} ({val_cycles - train_cycles}개)")
print(f"Test Cycles: {val_threshold_cycle+1} ~ {cycle_list[-1]} ({total_cycles - val_cycles}개)")

# 1. Train / Validation / Test split
train_df = df[df['cycle_idx'] <= train_threshold_cycle].copy()
val_df = df[(df['cycle_idx'] > train_threshold_cycle) & (df['cycle_idx'] <= val_threshold_cycle)].copy()
test_df = df[df['cycle_idx'] > val_threshold_cycle].copy()

print(f"\nTrain size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


# 2. Feature 준비

features1 = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load', 'Voltage_measured_smooth',
    'Voltage_measured_residual', 'Voltage_measured_trend',
    'Current_measured_smooth', 'Current_measured_residual',
    'Current_measured_trend', 'Temperature_measured_smooth',
    'Temperature_measured_residual', 'Temperature_measured_trend',
    'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
    'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'
]

X_train = train_df[features1]
X_val = val_df[features1]
X_test = test_df[features1]


# 3. Scaler: Train만으로 학습

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n Scaler 학습 완료 (Train 기준)")


# 4. LOF: Train만으로 학습

lof = LocalOutlierFactor(
    n_neighbors=30,
    contamination='auto',
    novelty=True
)

lof.fit(X_train_scaled)

print("LOF 모델 학습 완료 (Train 기준)")


# 5. 점수 계산 (Train/Validation/Test)

train_scores = -lof.negative_outlier_factor_
val_scores = -lof.score_samples(X_val_scaled)
test_scores = -lof.score_samples(X_test_scaled)

print(f"\n LOF 점수 범위:")
print(f"   Train: {train_scores.min():.4f} ~ {train_scores.max():.4f}")
print(f"   Validation: {val_scores.min():.4f} ~ {val_scores.max():.4f}")
print(f"   Test: {test_scores.min():.4f} ~ {test_scores.max():.4f}")


# 6. Validation으로 Quantile 기반 Threshold 찾기

# 여러 quantile 시도
quantiles = [0.99, 0.995, 0.999]

print("\n" + "="*70)
print(" Quantile 기반 Threshold 비교 (Validation 기준)")
print("="*70)

quantile_results = []

for q in quantiles:
    # Threshold 계산
    thr = np.quantile(val_scores, q)
    
    # 각 데이터셋의 이상치 개수
    val_anom = (val_scores >= thr).sum()
    test_anom = (test_scores >= thr).sum()
    
    quantile_results.append({
        'quantile': q,
        'threshold': thr,
        'val_count': val_anom,
        'val_pct': val_anom / len(val_df) * 100,
        'test_count': test_anom,
        'test_pct': test_anom / len(test_df) * 100
    })
    
    print(f"\nQuantile {q:.3f} (상위 {(1-q)*100:.1f}%):")
    print(f"   Threshold: {thr:.4f}")
    print(f"   Validation 이상치개수/비율: {val_anom}개 ({val_anom/len(val_df)*100:.2f}%)")
    print(f"   Test 이상치개수/비율: {test_anom}개 ({test_anom/len(test_df)*100:.2f}%)")





총 Cycle 수: 168
Train Cycles: 0 ~ 352 (100개)
Validation Cycles: 353 ~ 483 (34개)
Test Cycles: 484 ~ 614 (34개)

Train size: 29361
Validation size: 10664
Test size: 10260

 Scaler 학습 완료 (Train 기준)
LOF 모델 학습 완료 (Train 기준)

 LOF 점수 범위:
   Train: 0.9388 ~ 20.5561
   Validation: 0.9384 ~ 4.2310
   Test: 0.9356 ~ 7.6131

 Quantile 기반 Threshold 비교 (Validation 기준)

Quantile 0.990 (상위 1.0%):
   Threshold: 2.3324
   Validation 이상치개수/비율: 107개 (1.00%)
   Test 이상치개수/비율: 2771개 (27.01%)

Quantile 0.995 (상위 0.5%):
   Threshold: 2.5594
   Validation 이상치개수/비율: 54개 (0.51%)
   Test 이상치개수/비율: 1944개 (18.95%)

Quantile 0.999 (상위 0.1%):
   Threshold: 3.5380
   Validation 이상치개수/비율: 11개 (0.10%)
   Test 이상치개수/비율: 54개 (0.53%)
